# Introduction to DiCE for explaining classification models

### Objective
To generate counterfactuals, DiCE implements two kinds of methods: model-agnostic and gradient-based. 

* **Model-Agnostic**: These methods apply to any black-box classifier or regressor. Currently supported methods are:
    * Randomized Search
    * Genetic Search
    * KD Tree Search (for counterfactuals from a given training dataset)
* **Gradient-Based**: These methods apply to differentiable models, such as those returned by deep learning libraries like tensorflow and pytorch. 
* DiCE requires two inputs: a training dataset and a pre-trained ML model. 


### Installing DICE

DiCE supports Python 3+. The stable version of DiCE is available on PyPI. DiCE requires the following packages:

- jsonschema
- numpy
- scikit-learn
- pandas
- h5py
- tqdm
- [optional] tensorflow/pytorch (works with Tensorflow>=1.13)

In [2]:
#pip install dice-ml

In [2]:
# Sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Tensorflow import
import tensorflow as tf


# DiCE imports
import dice_ml
from dice_ml.utils import helpers  # helper functions

In [3]:
%load_ext autoreload
%autoreload 2

## Preliminaries: Loading a dataset and a ML model trained over it
### Loading the `Adult` dataset

In [4]:
dataset = helpers.load_adult_income_dataset()

In [5]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


Split the dataset into train and test sets.

In [7]:
target = dataset["income"]
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

In [8]:
# Step 1: dice_ml.Data
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### Loading the ML model

DiCE supports sklearn, tensorflow and pytorch models. 

In [9]:
numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

## Generating counterfactual examples using DiCE


In [10]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [11]:
e1 = exp.generate_counterfactuals(x_test[0:1], total_CFs=2, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.65it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Some-college,-,Professional,-,-,-,1
1,-,Government,-,Single,-,-,-,-,1


In [12]:
e1.visualize_as_dataframe(show_only_changes=False)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,Some-college,Married,Professional,White,Female,38,1
1,29,Government,HS-grad,Single,Blue-Collar,White,Female,38,1


In [13]:
# Changing only age and education
e2 = exp.generate_counterfactuals(x_test[0:1],
                                  total_CFs=2,
                                  desired_class="opposite",
                                  features_to_vary=["education", "occupation"]
                                  )
e2.visualize_as_dataframe(show_only_changes=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.24it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Assoc,-,Service,-,-,-,1
1,-,-,Doctorate,-,White-Collar,-,-,-,1


In [14]:
# Restricting age to be between [20,30] and Education to be either {'Doctorate', 'Prof-school'}.
e3 = exp.generate_counterfactuals(x_test[0:1],
                                  total_CFs=2,
                                  desired_class="opposite",
                                  permitted_range={'age': [20, 30], 'education': ['Doctorate', 'Prof-school']})
e3.visualize_as_dataframe(show_only_changes=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.12it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Prof-school,-,-,-,Male,-,1
1,-,-,Prof-school,-,Professional,-,-,-,1


## Generating feature attributions (local and global) using DiCE


### Local feature importance scores

These scores are computed for a given query instance (input point) by summarizing a set of counterfactual examples around the point. 

In [15]:
query_instance = x_test[0:1]
imp = exp.local_feature_importance(query_instance, total_CFs=10)
print(imp.local_importance)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.66it/s]

[{'education': 0.8, 'occupation': 0.5, 'workclass': 0.3, 'gender': 0.2, 'marital_status': 0.1, 'race': 0.1, 'age': 0.0, 'hours_per_week': 0.0}]


The `total_CFs` parameter denotes the number of counterfactuals that are used to create the local importance. More the better.

### Global feature importance scores

A global importance score per feature can be estimated by aggregating the scores over individual inputs. The more the inputs, the better the estimate for global importance of a feature.

In [16]:
query_instances = x_test[0:20]
imp = exp.global_feature_importance(query_instances)
print(imp.summary_importance)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.29it/s]

{'education': 0.545, 'occupation': 0.34, 'marital_status': 0.27, 'hours_per_week': 0.235, 'workclass': 0.205, 'age': 0.175, 'race': 0.135, 'gender': 0.11}


## Working with deep learning models (TensorFlow and PyTorch)

### Explaining a Tensorflow model

In [17]:
# supress deprecation warnings from TF
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

backend = 'TF'+tf.__version__[0]  # TF1
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
# Step 2: dice_ml.Model
m = dice_ml.Model(model_path=ML_modelpath, backend=backend)

In [18]:
# Step 3: initiate DiCE
exp = dice_ml.Dice(d, m)

Below we provide query instance as a dict.

In [19]:
# query instance in the form of a dictionary or a dataframe; keys: feature name, values: feature value
query_instance = {'age': 22,
                  'workclass': 'Private',
                  'education': 'HS-grad',
                  'marital_status': 'Single',
                  'occupation': 'Service',
                  'race': 'White',
                  'gender': 'Female',
                  'hours_per_week': 45}

In [20]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 44 sec


In [21]:
# visualize the result, highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.019



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,70.0,-,Masters,-,White-Collar,-,-,51.0,0
1,-,Self-Employed,Doctorate,Married,-,-,-,-,0
2,47.0,-,-,Married,-,-,-,-,0
3,36.0,-,Prof-school,Married,-,-,-,62.0,0


### Explaining a Pytorch model

Just change the backend variable to 'PYT' to use DiCE with PyTorch. Below, we use a pre-trained ML model in PyTorch which produces high accuracy comparable to other baselines. For convenience, we include the sample trained model with the DiCE package.

In [24]:
backend = 'PYT'
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
m = dice_ml.Model(model_path=ML_modelpath, backend=backend)

Instantiate the DiCE class with the new PyTorch model object *m*. 

In [25]:
exp = dice_ml.Dice(d, m)

In [26]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age': 22,
                  'workclass': 'Private',
                  'education': 'HS-grad',
                  'marital_status': 'Single',
                  'occupation': 'Service',
                  'race': 'White',
                  'gender': 'Female',
                  'hours_per_week': 45}

In [27]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 12 sec


In [28]:
# highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,72.0,-,-,Married,White-Collar,-,-,-,-
1,29.0,-,Prof-school,Married,-,-,Male,-,-
2,52.0,-,Doctorate,Married,-,-,-,-,-
3,47.0,-,Masters,Married,-,-,-,73.0,-


## More resources: 

DiCE has multiple configurable options and support for different kinds of models. Follow these notebooks to learn more.

1. You can constrain the features to vary, weigh the relative importance of different features for computing distance, or specify permitted ranges for each feature. Check out the [Customizing Counterfactuals Notebook](DiCE_with_advanced_options.ipynb). 
2. You can use it for multi-class classification or regression models. [Counterfactuals for Multi-class Classification and Regression Models Notebook](DiCE_multiclass_classification_and_regression.ipynb).
3. Explore the different model-agnostic explanation methods in DiCE. [Model-agnostic Counterfactual Generation Methods](DiCE_model_agnostic_CFs.ipynb).
4. You can generate CFs even without access to training data (e.g., for privacy-sensitive data). [DiCE with Private Data Notebook](DiCE_with_private_data.ipynb).
5. Feasibility of counterfactuals is an important consideration. You can try out this VAE-based method that adds a CF likelihood term to the loss. [VAE-based Counterfactuals (Pytorch only)](DiCE_getting_started_feasible.ipynb).

